In [28]:
#WEST BENGAL
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import pandas as pd

# URL of the website
URL = "https://www.redbus.in/online-booking/west-bengal-transport-corporation?utm_source=rtchometile"

# Set up the Chrome driver
driver = webdriver.Chrome()
driver.get(URL)
driver.maximize_window()
time.sleep(5)  # Wait for the page to load

# Function to scrape bus routes
def scrape_bus_routes():
    route_elements = driver.find_elements(By.CLASS_NAME, 'route')
    bus_routes_link = [route.get_attribute('href') for route in route_elements]
    bus_routes_name = [route.text.strip() for route in route_elements]
    return bus_routes_link, bus_routes_name

# Scrape the first page
all_bus_routes_link, all_bus_routes_name = scrape_bus_routes()

# Function to scrape bus details
def scrape_bus_details(url, route_name):
    try:
        driver.get(url)
        time.sleep(5)  # Allow the page to load
        
        # Scroll down to load all bus items
        last_height = driver.execute_script("return document.body.scrollHeight")
        
        while True:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(3)  # Wait for the page to load more content
            
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height
        
        # Find bus item details
        bus_name_elements = driver.find_elements(By.CLASS_NAME, "travels")
        bus_type_elements = driver.find_elements(By.CLASS_NAME, "bus-type")
        departing_time_elements = driver.find_elements(By.CLASS_NAME, "dp-time")
        duration_elements = driver.find_elements(By.CLASS_NAME, "dur")
        reaching_time_elements = driver.find_elements(By.CLASS_NAME, "bp-time")
        star_rating_elements = driver.find_elements(By.XPATH, "//div[@class='rating-sec lh-24']")
        price_elements = driver.find_elements(By.CLASS_NAME, "fare")
        seat_availability_elements = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left m-top-30') or contains(@class, 'seat-left m-top-16')]")

        bus_details = []
        for i in range(len(bus_name_elements)):
            bus_detail = {
                "Route_Name": route_name,
                "Route_Link": url,
                "Bus_Name": bus_name_elements[i].text,
                "Bus_Type": bus_type_elements[i].text,
                "Departing_Time": departing_time_elements[i].text,
                "Duration": duration_elements[i].text,
                "Reaching_Time": reaching_time_elements[i].text,
                "Star_Rating": star_rating_elements[i].text if i < len(star_rating_elements) else '0',
                "Price": price_elements[i].text,
                "Seat_Availability": seat_availability_elements[i].text if i < len(seat_availability_elements) else '0'
            }
            bus_details.append(bus_detail)
        return bus_details

    except Exception as e:
        print(f"Error occurred while accessing {url}: {str(e)}")
        return []

# List to hold all bus details
all_bus_details = []

# Iterate over each bus route link and scrape the details
for link, name in zip(all_bus_routes_link, all_bus_routes_name):
    bus_details = scrape_bus_details(link, name)
    if bus_details:
        all_bus_details.extend(bus_details)

# Convert the list of dictionaries to a DataFrame
df = pd.DataFrame(all_bus_details)

# Save the DataFrame to a CSV file
df.to_csv('wb2_bus_details.csv', index=False)

# Close the driver
driver.quit()

In [30]:
#TELANGANA
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

# URL of the website
URL = "https://www.redbus.in/online-booking/tsrtc/?utm_source=rtchometile"

def initialize_driver():
    driver = webdriver.Chrome()
    driver.maximize_window()
    return driver

def load_page(driver, url):
    driver.get(url)
    time.sleep(5)  # Wait for the page to load

# Function to scrape bus routes
def scrape_bus_routes(driver):
    route_elements = driver.find_elements(By.CLASS_NAME, 'route')
    bus_routes_link = [route.get_attribute('href') for route in route_elements]
    bus_routes_name = [route.text.strip() for route in route_elements]
    return bus_routes_link, bus_routes_name

# Function to scrape bus details
def scrape_bus_details(driver, url, route_name):
    try:
        driver.get(url)
        time.sleep(2)  # Allow the page to load
        
        # Click the "View Buses" button if it exists
        try:
            view_buses_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CLASS_NAME, "button"))
            )
            driver.execute_script("arguments[0].click();", view_buses_button)
            time.sleep(5)  # Wait for buses to load
            
            # Scroll down to load all bus items
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(5)  # Wait for the page to load more content

            # Find bus item details
            bus_name_elements = driver.find_elements(By.CLASS_NAME, "travels")
            bus_type_elements = driver.find_elements(By.CLASS_NAME, "bus-type")
            departing_time_elements = driver.find_elements(By.CLASS_NAME, "dp-time")
            duration_elements = driver.find_elements(By.CLASS_NAME, "dur")
            reaching_time_elements = driver.find_elements(By.CLASS_NAME, "bp-time")
            star_rating_elements = driver.find_elements(By.XPATH, "//div[@class='rating-sec lh-24']")
            price_elements = driver.find_elements(By.CLASS_NAME, "fare")

            # Use XPath to handle both seat availability classes
            seat_availability_elements = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left m-top-30') or contains(@class, 'seat-left m-top-16')]")

            bus_details = []
            for i in range(len(bus_name_elements)):
                bus_detail = {
                    "Route_Name": route_name,
                    "Route_Link": url,
                    "Bus_Name": bus_name_elements[i].text,
                    "Bus_Type": bus_type_elements[i].text,
                    "Departing_Time": departing_time_elements[i].text,
                    "Duration": duration_elements[i].text,
                    "Reaching_Time": reaching_time_elements[i].text,
                    "Star_Rating": star_rating_elements[i].text if i < len(star_rating_elements) else '0',
                    "Price": price_elements[i].text,
                    "Seat_Availability": seat_availability_elements[i].text if i < len(seat_availability_elements) else '0'
                }
                bus_details.append(bus_detail)
            return bus_details
        
        except Exception as e:
            print(f"Error occurred while scraping bus details for {url}: {str(e)}")
            return []

    except Exception as e:
        print(f"Error occurred while accessing {url}: {str(e)}")
        return []

# List to hold all bus details
all_bus_details = []

# Function to scrape all pages
def scrape_all_pages():
    for page in range(1, 4):  # There are 3 pages
        try:
            driver = initialize_driver()
            load_page(driver, URL)
            
            if page > 1:
                pagination_tab = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs')][text()='{page}']"))
                )
                driver.execute_script("arguments[0].scrollIntoView();", pagination_tab)
                driver.execute_script("arguments[0].click();", pagination_tab)
                time.sleep(5)  # Wait for the page to load
            
            all_bus_routes_link, all_bus_routes_name = scrape_bus_routes(driver)
            # Iterate over each bus route link and scrape the details
            for link, name in zip(all_bus_routes_link, all_bus_routes_name):
                bus_details = scrape_bus_details(driver, link, name)
                if bus_details:
                    all_bus_details.extend(bus_details)


        except Exception as e:
            print(f"Error occurred while accessing page {page}: {str(e)}")

# Scrape routes and details from all pages
scrape_all_pages()

# Convert the list of dictionaries to a DataFrame
df = pd.DataFrame(all_bus_details)

# Save the DataFrame to a CSV file
df.to_csv('Telangana_bus_details.csv', index=False)

driver.quit()

In [31]:
df

,Route_Name,Route_Link,Bus_Name,Bus_Type,Departing_Time,Duration,Reaching_Time,Star_Rating,Price,Seat_Availability
0,Khammam to Hyderabad,https://www.redbus.in/bus-tickets/khammam-to-h...,TSRTC - 1319,LAHARI A/C SLEEPER CUM SEATER,17:10,03h 35m,20:45,3.3,INR 389,15 Seats available
1,Khammam to Hyderabad,https://www.redbus.in/bus-tickets/khammam-to-h...,TSRTC - 8613,"SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)",17:20,04h 10m,21:30,3.7,INR 320,20 Seats available
2,Khammam to Hyderabad,https://www.redbus.in/bus-tickets/khammam-to-h...,TSRTC - 8675,RAJDHANI (A.C. Semi Sleeper),17:30,04h 00m,21:30,3.8,INR 389,31 Seats available
3,Khammam to Hyderabad,https://www.redbus.in/bus-tickets/khammam-to-h...,TSRTC - 64089,RAJDHANI (A.C. Semi Sleeper),18:00,04h 00m,22:00,3.1,INR 389,35 Seats available
4,Khammam to Hyderabad,https://www.redbus.in/bus-tickets/khammam-to-h...,TSRTC - 1952,LAHARI A/C SLEEPER CUM SEATER,18:10,03h 35m,21:45,3.1,INR 389,38 Seats available
...,...,...,...,...,...,...,...,...,...,...
472,Hyderabad to Tirupati,https://www.redbus.in/bus-tickets/hyderabad-to...,IntrCity SmartBus,AC Sleeper (2+1),18:35,12h 45m,07:20,4.7,INR 1147,8 Seats available
473,Hyderabad to Tirupati,https://www.redbus.in/bus-tickets/hyderabad-to...,Ramana Tours And Travels,A/C Sleeper (2+1),19:55,12h 40m,08:35,4.6,1600,6 Seats available
474,Hyderabad to Tirupati,https://www.redbus.in/bus-tickets/hyderabad-to...,Jagan Travels,A/C Seater / Sleeper (2+1),22:00,11h 30m,09:30,4.6,INR 800,5 Seats available
475,Hyderabad to Tirupati,https://www.redbus.in/bus-tickets/hyderabad-to...,Vikram Travels,A/C Sleeper (2+1),20:20,12h 10m,08:30,4.6,INR 1099,6 Seats available


In [35]:
#Rajasthan
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

# URL of the website
URL = "https://www.redbus.in/online-booking/rsrtc/?utm_source=rtchometile"

def initialize_driver():
    driver = webdriver.Chrome()
    driver.maximize_window()
    return driver

def load_page(driver, url):
    driver.get(url)
    time.sleep(5)  # Wait for the page to load

# Function to scrape bus routes
def scrape_bus_routes(driver):
    route_elements = driver.find_elements(By.CLASS_NAME, 'route')
    bus_routes_link = [route.get_attribute('href') for route in route_elements]
    bus_routes_name = [route.text.strip() for route in route_elements]
    return bus_routes_link, bus_routes_name

# Function to scrape bus details
def scrape_bus_details(driver, url, route_name):
    try:
        driver.get(url)
        time.sleep(2)  # Allow the page to load
        
        # Click the "View Buses" button if it exists
        try:
            view_buses_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CLASS_NAME, "button"))
            )
            driver.execute_script("arguments[0].click();", view_buses_button)
            time.sleep(5)  # Wait for buses to load
            
            # Scroll down to load all bus items
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(5)  # Wait for the page to load more content

            # Find bus item details
            bus_name_elements = driver.find_elements(By.CLASS_NAME, "travels")
            bus_type_elements = driver.find_elements(By.CLASS_NAME, "bus-type")
            departing_time_elements = driver.find_elements(By.CLASS_NAME, "dp-time")
            duration_elements = driver.find_elements(By.CLASS_NAME, "dur")
            reaching_time_elements = driver.find_elements(By.CLASS_NAME, "bp-time")
            star_rating_elements = driver.find_elements(By.XPATH, "//div[@class='rating-sec lh-24']")
            price_elements = driver.find_elements(By.CLASS_NAME, "fare")

            # Use XPath to handle both seat availability classes
            seat_availability_elements = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left m-top-30') or contains(@class, 'seat-left m-top-16')]")

            bus_details = []
            for i in range(len(bus_name_elements)):
                bus_detail = {
                    "Route_Name": route_name,
                    "Route_Link": url,
                    "Bus_Name": bus_name_elements[i].text,
                    "Bus_Type": bus_type_elements[i].text,
                    "Departing_Time": departing_time_elements[i].text,
                    "Duration": duration_elements[i].text,
                    "Reaching_Time": reaching_time_elements[i].text,
                    "Star_Rating": star_rating_elements[i].text if i < len(star_rating_elements) else '0',
                    "Price": price_elements[i].text,
                    "Seat_Availability": seat_availability_elements[i].text if i < len(seat_availability_elements) else '0'
                }
                bus_details.append(bus_detail)
            return bus_details
        
        except Exception as e:
            print(f"Error occurred while scraping bus details for {url}: {str(e)}")
            return []

    except Exception as e:
        print(f"Error occurred while accessing {url}: {str(e)}")
        return []

# List to hold all bus details
all_bus_details = []

# Function to scrape all pages
def scrape_all_pages():
    for page in range(1, 4):  # There are 3 pages
        try:
            driver = initialize_driver()
            load_page(driver, URL)
            
            if page > 1:
                pagination_tab = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs')][text()='{page}']"))
                )
                driver.execute_script("arguments[0].scrollIntoView();", pagination_tab)
                driver.execute_script("arguments[0].click();", pagination_tab)
                time.sleep(2)  # Wait for the page to load
            
            all_bus_routes_link, all_bus_routes_name = scrape_bus_routes(driver)
            # Iterate over each bus route link and scrape the details
            for link, name in zip(all_bus_routes_link, all_bus_routes_name):
                bus_details = scrape_bus_details(driver, link, name)
                if bus_details:
                    all_bus_details.extend(bus_details)


        except Exception as e:
            print(f"Error occurred while accessing page {page}: {str(e)}")

# Scrape routes and details from all pages
scrape_all_pages()

# Convert the list of dictionaries to a DataFrame
df = pd.DataFrame(all_bus_details)

# Save the DataFrame to a CSV file
df.to_csv('rajasthan_bus_details.csv', index=False)

# Close the driver
driver.quit()

In [34]:
df

,Route_Name,Route_Link,Bus_Name,Bus_Type,Departing_Time,Duration,Reaching_Time,Star_Rating,Price,Seat_Availability
0,Khammam to Hyderabad,https://www.redbus.in/bus-tickets/khammam-to-h...,TSRTC - 1319,LAHARI A/C SLEEPER CUM SEATER,17:10,03h 35m,20:45,3.3,INR 389,15 Seats available
1,Khammam to Hyderabad,https://www.redbus.in/bus-tickets/khammam-to-h...,TSRTC - 8613,"SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)",17:20,04h 10m,21:30,3.7,INR 320,20 Seats available
2,Khammam to Hyderabad,https://www.redbus.in/bus-tickets/khammam-to-h...,TSRTC - 8675,RAJDHANI (A.C. Semi Sleeper),17:30,04h 00m,21:30,3.8,INR 389,31 Seats available
3,Khammam to Hyderabad,https://www.redbus.in/bus-tickets/khammam-to-h...,TSRTC - 64089,RAJDHANI (A.C. Semi Sleeper),18:00,04h 00m,22:00,3.1,INR 389,35 Seats available
4,Khammam to Hyderabad,https://www.redbus.in/bus-tickets/khammam-to-h...,TSRTC - 1952,LAHARI A/C SLEEPER CUM SEATER,18:10,03h 35m,21:45,3.1,INR 389,38 Seats available
...,...,...,...,...,...,...,...,...,...,...
472,Hyderabad to Tirupati,https://www.redbus.in/bus-tickets/hyderabad-to...,IntrCity SmartBus,AC Sleeper (2+1),18:35,12h 45m,07:20,4.7,INR 1147,8 Seats available
473,Hyderabad to Tirupati,https://www.redbus.in/bus-tickets/hyderabad-to...,Ramana Tours And Travels,A/C Sleeper (2+1),19:55,12h 40m,08:35,4.6,1600,6 Seats available
474,Hyderabad to Tirupati,https://www.redbus.in/bus-tickets/hyderabad-to...,Jagan Travels,A/C Seater / Sleeper (2+1),22:00,11h 30m,09:30,4.6,INR 800,5 Seats available
475,Hyderabad to Tirupati,https://www.redbus.in/bus-tickets/hyderabad-to...,Vikram Travels,A/C Sleeper (2+1),20:20,12h 10m,08:30,4.6,INR 1099,6 Seats available


In [38]:
#Andrapradesh
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

# URL of the website
URL = "https://www.redbus.in/online-booking/apsrtc/?utm_source=rtchometile"

def initialize_driver():
    driver = webdriver.Chrome()
    driver.maximize_window()
    return driver

def load_page(driver, url):
    driver.get(url)
    time.sleep(2)  # Wait for the page to load

# Function to scrape bus routes
def scrape_bus_routes(driver):
    route_elements = driver.find_elements(By.CLASS_NAME, 'route')
    bus_routes_link = [route.get_attribute('href') for route in route_elements]
    bus_routes_name = [route.text.strip() for route in route_elements]
    return bus_routes_link, bus_routes_name

# Function to scrape bus details
def scrape_bus_details(driver, url, route_name):
    try:
        driver.get(url)
        time.sleep(2)  # Allow the page to load
        
        # Click the "View Buses" button if it exists
        try:
            view_buses_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CLASS_NAME, "button"))
            )
            driver.execute_script("arguments[0].click();", view_buses_button)
            time.sleep(2)  # Wait for buses to load
            
            # Scroll down to load all bus items
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(2)  # Wait for the page to load more content

            # Find bus item details
            bus_name_elements = driver.find_elements(By.CLASS_NAME, "travels")
            bus_type_elements = driver.find_elements(By.CLASS_NAME, "bus-type")
            departing_time_elements = driver.find_elements(By.CLASS_NAME, "dp-time")
            duration_elements = driver.find_elements(By.CLASS_NAME, "dur")
            reaching_time_elements = driver.find_elements(By.CLASS_NAME, "bp-time")
            star_rating_elements = driver.find_elements(By.XPATH, "//div[@class='rating-sec lh-24']")
            price_elements = driver.find_elements(By.CLASS_NAME, "fare")

            # Use XPath to handle both seat availability classes
            seat_availability_elements = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left m-top-30') or contains(@class, 'seat-left m-top-16')]")

            bus_details = []
            for i in range(len(bus_name_elements)):
                bus_detail = {
                    "Route_Name": route_name,
                    "Route_Link": url,
                    "Bus_Name": bus_name_elements[i].text,
                    "Bus_Type": bus_type_elements[i].text,
                    "Departing_Time": departing_time_elements[i].text,
                    "Duration": duration_elements[i].text,
                    "Reaching_Time": reaching_time_elements[i].text,
                    "Star_Rating": star_rating_elements[i].text if i < len(star_rating_elements) else '0',
                    "Price": price_elements[i].text,
                    "Seat_Availability": seat_availability_elements[i].text if i < len(seat_availability_elements) else '0'
                }
                bus_details.append(bus_detail)
            return bus_details
        
        except Exception as e:
            print(f"Error occurred while scraping bus details for {url}: {str(e)}")
            return []

    except Exception as e:
        print(f"Error occurred while accessing {url}: {str(e)}")
        return []

# List to hold all bus details
all_bus_details = []

# Function to scrape all pages
def scrape_all_pages():
    for page in range(1, 6):  # There are 5 pages
        try:
            driver = initialize_driver()
            load_page(driver, URL)
            
            if page > 1:
                pagination_tab = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs')][text()='{page}']"))
                )
                driver.execute_script("arguments[0].scrollIntoView();", pagination_tab)
                driver.execute_script("arguments[0].click();", pagination_tab)
                time.sleep(5)  # Wait for the page to load
            
            all_bus_routes_link, all_bus_routes_name = scrape_bus_routes(driver)
            # Iterate over each bus route link and scrape the details
            for link, name in zip(all_bus_routes_link, all_bus_routes_name):
                bus_details = scrape_bus_details(driver, link, name)
                if bus_details:
                    all_bus_details.extend(bus_details)
            

        except Exception as e:
            print(f"Error occurred while accessing page {page}: {str(e)}")

# Scrape routes and details from all pages
scrape_all_pages()

# Convert the list of dictionaries to a DataFrame
df = pd.DataFrame(all_bus_details)

# Save the DataFrame to a CSV file
df.to_csv('ap_bus_details.csv', index=False)

# Close the driver
driver.quit()
            

Error occurred while scraping bus details for https://www.redbus.in/bus-tickets/madanapalli-to-bangalore: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF6CB839642+30946]
	(No symbol) [0x00007FF6CB7EE3D9]
	(No symbol) [0x00007FF6CB6E6FDA]
	(No symbol) [0x00007FF6CB73822C]
	(No symbol) [0x00007FF6CB73850C]
	(No symbol) [0x00007FF6CB77DCB7]
	(No symbol) [0x00007FF6CB75CAAF]
	(No symbol) [0x00007FF6CB77B041]
	(No symbol) [0x00007FF6CB75C813]
	(No symbol) [0x00007FF6CB72A6E5]
	(No symbol) [0x00007FF6CB72B021]
	GetHandleVerifier [0x00007FF6CB96F84D+1301229]
	GetHandleVerifier [0x00007FF6CB97BDC7+1351783]
	GetHandleVerifier [0x00007FF6CB972A13+1313971]
	GetHandleVerifier [0x00007FF6CB86DD16+245686]
	(No symbol) [0x00007FF6CB7F759F]
	(No symbol) [0x00007FF6CB7F3814]
	(No symbol) [0x00007FF6CB7F39A2]
	(No symbol) [0x00007FF6CB7EA3FF]
	BaseThreadInitThunk [0x00007FFBF4AE257D+29]
	RtlUserThreadStart [0x00007FFBF642AF28+40]



In [37]:
df

,Route_Name,Route_Link,Bus_Name,Bus_Type,Departing_Time,Duration,Reaching_Time,Star_Rating,Price,Seat_Availability
0,Jodhpur to Ajmer,https://www.redbus.in/bus-tickets/jodhpur-to-a...,RSRTC - 193481,Express Non AC Seater 2+3,18:30,04h 30m,23:00,4.1,INR 229,42 Seats available
1,Jodhpur to Ajmer,https://www.redbus.in/bus-tickets/jodhpur-to-a...,RSRTC - 187140,NON AC Seater / Sleeper 2+1,19:30,04h 00m,23:30,4.5,INR 302,18 Seats available
2,Jodhpur to Ajmer,https://www.redbus.in/bus-tickets/jodhpur-to-a...,RSRTC - 176394,Super Luxury Volvo AC Seater Pushback 2+2,20:00,03h 45m,23:45,4.5,INR 460,23 Seats available
3,Jodhpur to Ajmer,https://www.redbus.in/bus-tickets/jodhpur-to-a...,RSRTC - 189473,NON AC Seater / Sleeper 2+1,21:00,04h 30m,01:30,4.1,INR 302,18 Seats available
4,Jodhpur to Ajmer,https://www.redbus.in/bus-tickets/jodhpur-to-a...,RSRTC - 192299,Super Luxury Volvo AC Seater Pushback 2+2,23:00,03h 45m,02:45,3.0,INR 458,37 Seats available
...,...,...,...,...,...,...,...,...,...,...
452,Bikaner to Sikar,https://www.redbus.in/bus-tickets/bikaner-to-s...,RSRTC - 182762,Express Non AC Seater 2+3,21:55,04h 50m,02:45,4.4,INR 231,42 Seats available
453,Bikaner to Sikar,https://www.redbus.in/bus-tickets/bikaner-to-s...,Milan Travels,Bharat Benz A/C Seater /Sleeper (2+1),17:20,03h 10m,20:30,4.5,INR 476,14 Seats available
454,Bikaner to Sikar,https://www.redbus.in/bus-tickets/bikaner-to-s...,Neelam Travels,A/C Seater / Sleeper (2+1),18:30,04h 30m,23:00,3.6,INR 800,6 Seats available
455,Bikaner to Sikar,https://www.redbus.in/bus-tickets/bikaner-to-s...,,,,,,,,


In [52]:
#Assam
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

# URL of the website
URL = "https://www.redbus.in/online-booking/astc/?utm_source=rtchometile"

def initialize_driver():
    driver = webdriver.Chrome()
    driver.maximize_window()
    return driver

def load_page(driver, url):
    driver.get(url)
    time.sleep(2)  # Wait for the page to load

# Function to scrape bus routes
def scrape_bus_routes(driver):
    route_elements = driver.find_elements(By.CLASS_NAME, 'route')
    bus_routes_link = [route.get_attribute('href') for route in route_elements]
    bus_routes_name = [route.text.strip() for route in route_elements]
    return bus_routes_link, bus_routes_name

# Function to scrape bus details
def scrape_bus_details(driver, url, route_name):
    try:
        driver.get(url)
        time.sleep(2)  # Allow the page to load
        
        # Click the "View Buses" button if it exists
        try:
            view_buses_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CLASS_NAME, "button"))
            )
            driver.execute_script("arguments[0].click();", view_buses_button)
            time.sleep(2)  # Wait for buses to load
            
            # Scroll down to load all bus items
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(2)  # Wait for the page to load more content

            # Find bus item details
            bus_name_elements = driver.find_elements(By.CLASS_NAME, "travels")
            bus_type_elements = driver.find_elements(By.CLASS_NAME, "bus-type")
            departing_time_elements = driver.find_elements(By.CLASS_NAME, "dp-time")
            duration_elements = driver.find_elements(By.CLASS_NAME, "dur")
            reaching_time_elements = driver.find_elements(By.CLASS_NAME, "bp-time")
            star_rating_elements = driver.find_elements(By.XPATH, "//div[@class='rating-sec lh-24']")
            price_elements = driver.find_elements(By.CLASS_NAME, "fare")

            # Use XPath to handle both seat availability classes
            seat_availability_elements = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left m-top-30') or contains(@class, 'seat-left m-top-16')]")

            bus_details = []
            for i in range(len(bus_name_elements)):
                bus_detail = {
                    "Route_Name": route_name,
                    "Route_Link": url,
                    "Bus_Name": bus_name_elements[i].text,
                    "Bus_Type": bus_type_elements[i].text,
                    "Departing_Time": departing_time_elements[i].text,
                    "Duration": duration_elements[i].text,
                    "Reaching_Time": reaching_time_elements[i].text,
                    "Star_Rating": star_rating_elements[i].text if i < len(star_rating_elements) else '0',
                    "Price": price_elements[i].text,
                    "Seat_Availability": seat_availability_elements[i].text if i < len(seat_availability_elements) else '0'
                }
                bus_details.append(bus_detail)
            return bus_details
        
        except Exception as e:
            print(f"Error occurred while scraping bus details for {url}: {str(e)}")
            return []

    except Exception as e:
        print(f"Error occurred while accessing {url}: {str(e)}")
        return []

# List to hold all bus details
all_bus_details = []

# Function to scrape all pages
def scrape_all_pages():
    for page in range(1, 6):  # There are 5 pages
        try:
            driver = initialize_driver()
            load_page(driver, URL)
            
            if page > 1:
                pagination_tab = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs')][text()='{page}']"))
                )
                driver.execute_script("arguments[0].scrollIntoView();", pagination_tab)
                driver.execute_script("arguments[0].click();", pagination_tab)
                time.sleep(5)  # Wait for the page to load
            
            all_bus_routes_link, all_bus_routes_name = scrape_bus_routes(driver)
            # Iterate over each bus route link and scrape the details
            for link, name in zip(all_bus_routes_link, all_bus_routes_name):
                bus_details = scrape_bus_details(driver, link, name)
                if bus_details:
                    all_bus_details.extend(bus_details)
            

        except Exception as e:
            print(f"Error occurred while accessing page {page}: {str(e)}")

# Scrape routes and details from all pages
scrape_all_pages()

# Convert the list of dictionaries to a DataFrame
df = pd.DataFrame(all_bus_details)

# Save the DataFrame to a CSV file
df.to_csv('assam_bus_details.csv', index=False)

# Close the driver
driver.quit()

Error occurred while scraping bus details for https://www.redbus.in/bus-tickets/nagaon-to-guwahati: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF6CB839642+30946]
	(No symbol) [0x00007FF6CB7EE3D9]
	(No symbol) [0x00007FF6CB6E6FDA]
	(No symbol) [0x00007FF6CB73822C]
	(No symbol) [0x00007FF6CB73850C]
	(No symbol) [0x00007FF6CB77DCB7]
	(No symbol) [0x00007FF6CB75CAAF]
	(No symbol) [0x00007FF6CB77B041]
	(No symbol) [0x00007FF6CB75C813]
	(No symbol) [0x00007FF6CB72A6E5]
	(No symbol) [0x00007FF6CB72B021]
	GetHandleVerifier [0x00007FF6CB96F84D+1301229]
	GetHandleVerifier [0x00007FF6CB97BDC7+1351783]
	GetHandleVerifier [0x00007FF6CB972A13+1313971]
	GetHandleVerifier [0x00007FF6CB86DD16+245686]
	(No symbol) [0x00007FF6CB7F759F]
	(No symbol) [0x00007FF6CB7F3814]
	(No symbol) [0x00007FF6CB7F39A2]
	(No symbol) [0x00007FF6CB7EA3FF]
	BaseThreadInitThunk [0x00007FFBF4AE257D+29]
	RtlUserThreadStart [0x00007FFBF642AF28+40]

Error occurred while scraping bus details for https://www.redbus.in/bus-tic

In [53]:
df

,Route_Name,Route_Link,Bus_Name,Bus_Type,Departing_Time,Duration,Reaching_Time,Star_Rating,Price,Seat_Availability
0,Tezpur to Guwahati,https://www.redbus.in/bus-tickets/tezpur-to-gu...,BLUE HILL TRAVELS INDIA LIMITED,NON A/C Seater Semi Sleeper (2+1),21:30,06h 00m,03:30,2.9,INR 400,8 Seats available
1,Guwahati to Tezpur,https://www.redbus.in/bus-tickets/guwahati-to-...,Yatra Travels,NON AC Seater/ Sleeper (2+1),20:00,04h 00m,00:00,4.1,400,13 Seats available
2,Guwahati to Tezpur,https://www.redbus.in/bus-tickets/guwahati-to-...,Yatra Travels,NON AC Seater/ Sleeper (2+1),20:30,04h 30m,01:00,3.8,350,27 Seats available
3,Guwahati to Tezpur,https://www.redbus.in/bus-tickets/guwahati-to-...,Pranjit Travels (Under ASTC),A/C Seater / Sleeper (2+1),20:45,05h 45m,02:30,3.3,INR 500,13 Seats available
4,Guwahati to Tezpur,https://www.redbus.in/bus-tickets/guwahati-to-...,Pranjit Travels (Under ASTC),NON A/C Seater (2+1),21:45,04h 15m,02:00,3.8,INR 450,14 Seats available
5,Guwahati to Tezpur,https://www.redbus.in/bus-tickets/guwahati-to-...,BLUE HILL TRAVELS INDIA LIMITED,NON A/C Seater Semi Sleeper (2+1),20:00,05h 15m,01:15,2.1,400,13 Seats available
6,Guwahati to Tezpur,https://www.redbus.in/bus-tickets/guwahati-to-...,Himalayan Travels,NON A/C Seater (2+2),20:30,06h 00m,02:30,3.1,450,32 Seats available
7,Guwahati to Tezpur,https://www.redbus.in/bus-tickets/guwahati-to-...,KIRAN TRAVELS,A/C Seater (2+1),20:40,03h 50m,00:30,4.1,540,22 Seats available
8,Guwahati to Tezpur,https://www.redbus.in/bus-tickets/guwahati-to-...,Pranjit Travels (Under ASTC),Bharat Benz A/C Seater (2+1),21:30,05h 00m,02:30,0,INR 600,10 Seats available
9,Dhubri to Guwahati,https://www.redbus.in/bus-tickets/dhubri-to-gu...,Yatra,Non AC Seater (2+2),20:15,07h 15m,03:30,2.6,INR 350,39 Seats available


In [42]:
#chandigarh
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

# URL of the website
URL = "https://www.redbus.in/online-booking/chandigarh-transport-undertaking-ctu"

def initialize_driver():
    driver = webdriver.Chrome()
    driver.maximize_window()
    return driver

def load_page(driver, url):
    driver.get(url)
    time.sleep(2)  # Wait for the page to load

# Function to scrape bus routes
def scrape_bus_routes(driver):
    route_elements = driver.find_elements(By.CLASS_NAME, 'route')
    bus_routes_link = [route.get_attribute('href') for route in route_elements]
    bus_routes_name = [route.text.strip() for route in route_elements]
    return bus_routes_link, bus_routes_name

# Function to scrape bus details
def scrape_bus_details(driver, url, route_name):
    try:
        driver.get(url)
        time.sleep(2)  # Allow the page to load
        
        # Click the "View Buses" button if it exists
        try:
            view_buses_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CLASS_NAME, "button"))
            )
            driver.execute_script("arguments[0].click();", view_buses_button)
            time.sleep(2)  # Wait for buses to load
            
            # Scroll down to load all bus items
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(2)  # Wait for the page to load more content

            # Find bus item details
            bus_name_elements = driver.find_elements(By.CLASS_NAME, "travels")
            bus_type_elements = driver.find_elements(By.CLASS_NAME, "bus-type")
            departing_time_elements = driver.find_elements(By.CLASS_NAME, "dp-time")
            duration_elements = driver.find_elements(By.CLASS_NAME, "dur")
            reaching_time_elements = driver.find_elements(By.CLASS_NAME, "bp-time")
            star_rating_elements = driver.find_elements(By.XPATH, "//div[@class='rating-sec lh-24']")
            price_elements = driver.find_elements(By.CLASS_NAME, "fare")

            # Use XPath to handle both seat availability classes
            seat_availability_elements = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left m-top-30') or contains(@class, 'seat-left m-top-16')]")

            bus_details = []
            for i in range(len(bus_name_elements)):
                bus_detail = {
                    "Route_Name": route_name,
                    "Route_Link": url,
                    "Bus_Name": bus_name_elements[i].text,
                    "Bus_Type": bus_type_elements[i].text,
                    "Departing_Time": departing_time_elements[i].text,
                    "Duration": duration_elements[i].text,
                    "Reaching_Time": reaching_time_elements[i].text,
                    "Star_Rating": star_rating_elements[i].text if i < len(star_rating_elements) else '0',
                    "Price": price_elements[i].text,
                    "Seat_Availability": seat_availability_elements[i].text if i < len(seat_availability_elements) else '0'
                }
                bus_details.append(bus_detail)
            return bus_details
        
        except Exception as e:
            print(f"Error occurred while scraping bus details for {url}: {str(e)}")
            return []

    except Exception as e:
        print(f"Error occurred while accessing {url}: {str(e)}")
        return []

# List to hold all bus details
all_bus_details = []

# Function to scrape all pages
def scrape_all_pages():
    for page in range(1, 6):  # There are 5 pages
        try:
            driver = initialize_driver()
            load_page(driver, URL)
            
            if page > 1:
                pagination_tab = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs')][text()='{page}']"))
                )
                driver.execute_script("arguments[0].scrollIntoView();", pagination_tab)
                driver.execute_script("arguments[0].click();", pagination_tab)
                time.sleep(5)  # Wait for the page to load
            
            all_bus_routes_link, all_bus_routes_name = scrape_bus_routes(driver)
            # Iterate over each bus route link and scrape the details
            for link, name in zip(all_bus_routes_link, all_bus_routes_name):
                bus_details = scrape_bus_details(driver, link, name)
                if bus_details:
                    all_bus_details.extend(bus_details)
            

        except Exception as e:
            print(f"Error occurred while accessing page {page}: {str(e)}")

# Scrape routes and details from all pages
scrape_all_pages()

# Convert the list of dictionaries to a DataFrame
df = pd.DataFrame(all_bus_details)

# Save the DataFrame to a CSV file
df.to_csv('chandigarh_bus_details.csv', index=False)

# Close the driver
driver.quit()

Error occurred while scraping bus details for https://www.redbus.in/bus-tickets/vrindavan-to-chandigarh: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF6CB839642+30946]
	(No symbol) [0x00007FF6CB7EE3D9]
	(No symbol) [0x00007FF6CB6E6FDA]
	(No symbol) [0x00007FF6CB73822C]
	(No symbol) [0x00007FF6CB73850C]
	(No symbol) [0x00007FF6CB77DCB7]
	(No symbol) [0x00007FF6CB75CAAF]
	(No symbol) [0x00007FF6CB77B041]
	(No symbol) [0x00007FF6CB75C813]
	(No symbol) [0x00007FF6CB72A6E5]
	(No symbol) [0x00007FF6CB72B021]
	GetHandleVerifier [0x00007FF6CB96F84D+1301229]
	GetHandleVerifier [0x00007FF6CB97BDC7+1351783]
	GetHandleVerifier [0x00007FF6CB972A13+1313971]
	GetHandleVerifier [0x00007FF6CB86DD16+245686]
	(No symbol) [0x00007FF6CB7F759F]
	(No symbol) [0x00007FF6CB7F3814]
	(No symbol) [0x00007FF6CB7F39A2]
	(No symbol) [0x00007FF6CB7EA3FF]
	BaseThreadInitThunk [0x00007FFBF4AE257D+29]
	RtlUserThreadStart [0x00007FFBF642AF28+40]

Error occurred while scraping bus details for https://www.redbus.in/bu

In [43]:
df

,Route_Name,Route_Link,Bus_Name,Bus_Type,Departing_Time,Duration,Reaching_Time,Star_Rating,Price,Seat_Availability
0,Chandigarh to Delhi,https://www.redbus.in/bus-tickets/chandigarh-t...,RSRTC - 192348,Super Luxury Volvo AC Seater Pushback 2+2,21:15,04h 45m,02:00,1.8,INR 676,27 Seats available
1,Chandigarh to Delhi,https://www.redbus.in/bus-tickets/chandigarh-t...,RSRTC - 166269,Express Non AC Seater 2+3,22:00,07h 55m,05:55,3.7,INR 320,42 Seats available
2,Chandigarh to Delhi,https://www.redbus.in/bus-tickets/chandigarh-t...,RSRTC - 166262,Express Non AC Seater 2+3,22:30,05h 55m,04:25,3.7,INR 320,39 Seats available
3,Chandigarh to Delhi,https://www.redbus.in/bus-tickets/chandigarh-t...,HRTC - 6639,Himsuta AC Seater Volvo/Scania 2+2,22:35,04h 55m,03:30,4.6,INR 523,11 Seats available
4,Chandigarh to Delhi,https://www.redbus.in/bus-tickets/chandigarh-t...,Vijay Tour and Travels,A/C Seater / Sleeper (2+1),17:55,05h 45m,23:40,3.5,555,25 Seats available
...,...,...,...,...,...,...,...,...,...,...
417,Baijnath to Chandigarh,https://www.redbus.in/bus-tickets/baijnath-to-...,Bharmani Travels,Volvo A/C Semi Sleeper (2+2),20:30,06h 00m,02:30,4.0,600,13 Seats available
418,Baijnath to Chandigarh,https://www.redbus.in/bus-tickets/baijnath-to-...,Northern Travels,Volvo A/C Semi Sleeper (2+2),18:00,07h 00m,01:00,2.8,INR 600,19 Seats available
419,Narnaul to Chandigarh,https://www.redbus.in/bus-tickets/narnaul-to-c...,Hari Das Tour &Travels,Bharat Benz A/C Seater /Sleeper (2+1),22:15,06h 05m,04:20,3.3,INR 599,15 Seats available
420,Chandigarh to Jawala Ji,https://www.redbus.in/bus-tickets/chandigarh-t...,HRTC - 573,Ordinary 3+2 Non AC Seater,19:40,04h 35m,00:15,3.9,INR 318,27 Seats available


In [44]:
#Himachal Pradesh
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

# URL of the website
URL = "https://www.redbus.in/online-booking/hrtc/?utm_source=rtchometile"

def initialize_driver():
    driver = webdriver.Chrome()
    driver.maximize_window()
    return driver

def load_page(driver, url):
    driver.get(url)
    time.sleep(2)  # Wait for the page to load

# Function to scrape bus routes
def scrape_bus_routes(driver):
    route_elements = driver.find_elements(By.CLASS_NAME, 'route')
    bus_routes_link = [route.get_attribute('href') for route in route_elements]
    bus_routes_name = [route.text.strip() for route in route_elements]
    return bus_routes_link, bus_routes_name

# Function to scrape bus details
def scrape_bus_details(driver, url, route_name):
    try:
        driver.get(url)
        time.sleep(2)  # Allow the page to load
        
        # Click the "View Buses" button if it exists
        try:
            view_buses_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CLASS_NAME, "button"))
            )
            driver.execute_script("arguments[0].click();", view_buses_button)
            time.sleep(2)  # Wait for buses to load
            
            # Scroll down to load all bus items
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(2)  # Wait for the page to load more content

            # Find bus item details
            bus_name_elements = driver.find_elements(By.CLASS_NAME, "travels")
            bus_type_elements = driver.find_elements(By.CLASS_NAME, "bus-type")
            departing_time_elements = driver.find_elements(By.CLASS_NAME, "dp-time")
            duration_elements = driver.find_elements(By.CLASS_NAME, "dur")
            reaching_time_elements = driver.find_elements(By.CLASS_NAME, "bp-time")
            star_rating_elements = driver.find_elements(By.XPATH, "//div[@class='rating-sec lh-24']")
            price_elements = driver.find_elements(By.CLASS_NAME, "fare")

            # Use XPath to handle both seat availability classes
            seat_availability_elements = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left m-top-30') or contains(@class, 'seat-left m-top-16')]")

            bus_details = []
            for i in range(len(bus_name_elements)):
                bus_detail = {
                    "Route_Name": route_name,
                    "Route_Link": url,
                    "Bus_Name": bus_name_elements[i].text,
                    "Bus_Type": bus_type_elements[i].text,
                    "Departing_Time": departing_time_elements[i].text,
                    "Duration": duration_elements[i].text,
                    "Reaching_Time": reaching_time_elements[i].text,
                    "Star_Rating": star_rating_elements[i].text if i < len(star_rating_elements) else '0',
                    "Price": price_elements[i].text,
                    "Seat_Availability": seat_availability_elements[i].text if i < len(seat_availability_elements) else '0'
                }
                bus_details.append(bus_detail)
            return bus_details
        
        except Exception as e:
            print(f"Error occurred while scraping bus details for {url}: {str(e)}")
            return []

    except Exception as e:
        print(f"Error occurred while accessing {url}: {str(e)}")
        return []

# List to hold all bus details
all_bus_details = []

# Function to scrape all pages
def scrape_all_pages():
    for page in range(1, 6):  # There are 5 pages
        try:
            driver = initialize_driver()
            load_page(driver, URL)
            
            if page > 1:
                pagination_tab = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs')][text()='{page}']"))
                )
                driver.execute_script("arguments[0].scrollIntoView();", pagination_tab)
                driver.execute_script("arguments[0].click();", pagination_tab)
                time.sleep(5)  # Wait for the page to load
            
            all_bus_routes_link, all_bus_routes_name = scrape_bus_routes(driver)
            # Iterate over each bus route link and scrape the details
            for link, name in zip(all_bus_routes_link, all_bus_routes_name):
                bus_details = scrape_bus_details(driver, link, name)
                if bus_details:
                    all_bus_details.extend(bus_details)


        except Exception as e:
            print(f"Error occurred while accessing page {page}: {str(e)}")

# Scrape routes and details from all pages
scrape_all_pages()

# Convert the list of dictionaries to a DataFrame
df = pd.DataFrame(all_bus_details)

# Save the DataFrame to a CSV file
df.to_csv('himachal_bus_details.csv', index=False)

# Close the driver
driver.quit()

Error occurred while scraping bus details for https://www.redbus.in/bus-tickets/chandigarh-to-hamirpur-himachal-pradesh: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF6CB839642+30946]
	(No symbol) [0x00007FF6CB7EE3D9]
	(No symbol) [0x00007FF6CB6E6FDA]
	(No symbol) [0x00007FF6CB73822C]
	(No symbol) [0x00007FF6CB73850C]
	(No symbol) [0x00007FF6CB77DCB7]
	(No symbol) [0x00007FF6CB75CAAF]
	(No symbol) [0x00007FF6CB77B041]
	(No symbol) [0x00007FF6CB75C813]
	(No symbol) [0x00007FF6CB72A6E5]
	(No symbol) [0x00007FF6CB72B021]
	GetHandleVerifier [0x00007FF6CB96F84D+1301229]
	GetHandleVerifier [0x00007FF6CB97BDC7+1351783]
	GetHandleVerifier [0x00007FF6CB972A13+1313971]
	GetHandleVerifier [0x00007FF6CB86DD16+245686]
	(No symbol) [0x00007FF6CB7F759F]
	(No symbol) [0x00007FF6CB7F3814]
	(No symbol) [0x00007FF6CB7F39A2]
	(No symbol) [0x00007FF6CB7EA3FF]
	BaseThreadInitThunk [0x00007FFBF4AE257D+29]
	RtlUserThreadStart [0x00007FFBF642AF28+40]

Error occurred while scraping bus details for https://

In [45]:
df

,Route_Name,Route_Link,Bus_Name,Bus_Type,Departing_Time,Duration,Reaching_Time,Star_Rating,Price,Seat_Availability
0,Delhi to Shimla,https://www.redbus.in/bus-tickets/delhi-to-shimla,HRTC - 93,Himsuta AC Seater Volvo/Scania 2+2,19:35,09h 10m,04:45,3.9,INR 892,23 Seats available
1,Delhi to Shimla,https://www.redbus.in/bus-tickets/delhi-to-shimla,HRTC - 556,Ordinary 3+2 Non AC Seater,20:10,09h 35m,05:45,4.5,INR 507,20 Seats available
2,Delhi to Shimla,https://www.redbus.in/bus-tickets/delhi-to-shimla,HRTC - 591,A/C Executive (2+3),21:00,10h 20m,07:20,4.0,INR 622,20 Seats available
3,Delhi to Shimla,https://www.redbus.in/bus-tickets/delhi-to-shimla,HRTC - 8,Himsuta AC Seater Volvo/Scania 2+2,21:02,09h 28m,06:30,4.4,INR 892,14 Seats available
4,Delhi to Shimla,https://www.redbus.in/bus-tickets/delhi-to-shimla,HRTC - 570,Ordinary 3+2 Non AC Seater,21:25,09h 20m,06:45,3.6,INR 507,25 Seats available
...,...,...,...,...,...,...,...,...,...,...
571,Ghumarwin to Delhi,https://www.redbus.in/bus-tickets/ghumarwin-to...,HRTC - 15,Himsuta AC Seater Volvo/Scania 2+2,21:25,07h 05m,04:30,3.7,INR 856,12 Seats available
572,Delhi to Sarkaghat,https://www.redbus.in/bus-tickets/delhi-to-sar...,HRTC - 541,Ordinary 3+2 Non AC Seater,19:14,08h 56m,04:10,4.8,INR 575,30 Seats available
573,Delhi to Sarkaghat,https://www.redbus.in/bus-tickets/delhi-to-sar...,HRTC - 549,A/C Executive (2+3),19:25,11h 20m,06:45,3.3,INR 701,30 Seats available
574,Delhi to Sarkaghat,https://www.redbus.in/bus-tickets/delhi-to-sar...,HRTC - 557,Ordinary 3+2 Non AC Seater,20:15,10h 55m,07:10,2.9,INR 575,27 Seats available


In [46]:
#Kerela
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

# URL of the website
URL = "https://www.redbus.in/online-booking/ksrtc-kerala/?utm_source=rtchometile"

def initialize_driver():
    driver = webdriver.Chrome()
    driver.maximize_window()
    return driver

def load_page(driver, url):
    driver.get(url)
    time.sleep(2)  # Wait for the page to load

# Function to scrape bus routes
def scrape_bus_routes(driver):
    route_elements = driver.find_elements(By.CLASS_NAME, 'route')
    bus_routes_link = [route.get_attribute('href') for route in route_elements]
    bus_routes_name = [route.text.strip() for route in route_elements]
    return bus_routes_link, bus_routes_name

# Function to scrape bus details
def scrape_bus_details(driver, url, route_name):
    try:
        driver.get(url)
        time.sleep(2)  # Allow the page to load
        
        # Click the "View Buses" button if it exists
        try:
            view_buses_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CLASS_NAME, "button"))
            )
            driver.execute_script("arguments[0].click();", view_buses_button)
            time.sleep(2)  # Wait for buses to load
            
            # Scroll down to load all bus items
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(2)  # Wait for the page to load more content

            # Find bus item details
            bus_name_elements = driver.find_elements(By.CLASS_NAME, "travels")
            bus_type_elements = driver.find_elements(By.CLASS_NAME, "bus-type")
            departing_time_elements = driver.find_elements(By.CLASS_NAME, "dp-time")
            duration_elements = driver.find_elements(By.CLASS_NAME, "dur")
            reaching_time_elements = driver.find_elements(By.CLASS_NAME, "bp-time")
            star_rating_elements = driver.find_elements(By.XPATH, "//div[@class='rating-sec lh-24']")
            price_elements = driver.find_elements(By.CLASS_NAME, "fare")

            # Use XPath to handle both seat availability classes
            seat_availability_elements = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left m-top-30') or contains(@class, 'seat-left m-top-16')]")

            bus_details = []
            for i in range(len(bus_name_elements)):
                bus_detail = {
                    "Route_Name": route_name,
                    "Route_Link": url,
                    "Bus_Name": bus_name_elements[i].text,
                    "Bus_Type": bus_type_elements[i].text,
                    "Departing_Time": departing_time_elements[i].text,
                    "Duration": duration_elements[i].text,
                    "Reaching_Time": reaching_time_elements[i].text,
                    "Star_Rating": star_rating_elements[i].text if i < len(star_rating_elements) else '0',
                    "Price": price_elements[i].text,
                    "Seat_Availability": seat_availability_elements[i].text if i < len(seat_availability_elements) else '0'
                }
                bus_details.append(bus_detail)
            return bus_details
        
        except Exception as e:
            print(f"Error occurred while scraping bus details for {url}: {str(e)}")
            return []

    except Exception as e:
        print(f"Error occurred while accessing {url}: {str(e)}")
        return []

# List to hold all bus details
all_bus_details = []

# Function to scrape all pages
def scrape_all_pages():
    for page in range(1, 3):  # There are 2 pages
        try:
            driver = initialize_driver()
            load_page(driver, URL)
            
            if page > 1:
                pagination_tab = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs')][text()='{page}']"))
                )
                driver.execute_script("arguments[0].scrollIntoView();", pagination_tab)
                driver.execute_script("arguments[0].click();", pagination_tab)
                time.sleep(5)  # Wait for the page to load
            
            all_bus_routes_link, all_bus_routes_name = scrape_bus_routes(driver)
            # Iterate over each bus route link and scrape the details
            for link, name in zip(all_bus_routes_link, all_bus_routes_name):
                bus_details = scrape_bus_details(driver, link, name)
                if bus_details:
                    all_bus_details.extend(bus_details)
            

        except Exception as e:
            print(f"Error occurred while accessing page {page}: {str(e)}")

# Scrape routes and details from all pages
scrape_all_pages()

# Convert the list of dictionaries to a DataFrame
df = pd.DataFrame(all_bus_details)

# Save the DataFrame to a CSV file
df.to_csv('kerala_bus_details.csv', index=False)

# Close the driver
driver.quit()

Error occurred while scraping bus details for https://www.redbus.in/bus-tickets/coimbatore-to-ooty: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF6CB839642+30946]
	(No symbol) [0x00007FF6CB7EE3D9]
	(No symbol) [0x00007FF6CB6E6FDA]
	(No symbol) [0x00007FF6CB73822C]
	(No symbol) [0x00007FF6CB73850C]
	(No symbol) [0x00007FF6CB77DCB7]
	(No symbol) [0x00007FF6CB75CAAF]
	(No symbol) [0x00007FF6CB77B041]
	(No symbol) [0x00007FF6CB75C813]
	(No symbol) [0x00007FF6CB72A6E5]
	(No symbol) [0x00007FF6CB72B021]
	GetHandleVerifier [0x00007FF6CB96F84D+1301229]
	GetHandleVerifier [0x00007FF6CB97BDC7+1351783]
	GetHandleVerifier [0x00007FF6CB972A13+1313971]
	GetHandleVerifier [0x00007FF6CB86DD16+245686]
	(No symbol) [0x00007FF6CB7F759F]
	(No symbol) [0x00007FF6CB7F3814]
	(No symbol) [0x00007FF6CB7F39A2]
	(No symbol) [0x00007FF6CB7EA3FF]
	BaseThreadInitThunk [0x00007FFBF4AE257D+29]
	RtlUserThreadStart [0x00007FFBF642AF28+40]



In [47]:
df

,Route_Name,Route_Link,Bus_Name,Bus_Type,Departing_Time,Duration,Reaching_Time,Star_Rating,Price,Seat_Availability
0,Bangalore to Kozhikode,https://www.redbus.in/bus-tickets/bangalore-to...,KSRTC (Kerala) - 1306,Super Express Non AC Seater Air Bus (2+2),21:46,09h 15m,07:01,3.7,INR 832,25 Seats available
1,Bangalore to Kozhikode,https://www.redbus.in/bus-tickets/bangalore-to...,MMK Travels,VE A/C Seater / Sleeper (2+1),22:30,09h 00m,07:30,4.4,INR 700,13 Seats available
2,Bangalore to Kozhikode,https://www.redbus.in/bus-tickets/bangalore-to...,NS Transports,Non A/C Seater / Sleeper (2+1),21:45,08h 25m,06:10,4.2,INR 900,7 Seats available
3,Bangalore to Kozhikode,https://www.redbus.in/bus-tickets/bangalore-to...,AdSuraj Holidays,VE A/C Sleeper (2+1),22:30,09h 00m,07:30,3.9,INR 1300,10 Seats available
4,Bangalore to Kozhikode,https://www.redbus.in/bus-tickets/bangalore-to...,P K Travels,A/C Sleeper (2+1),19:15,11h 15m,06:30,4.4,INR 1500,5 Seats available
...,...,...,...,...,...,...,...,...,...,...
385,Kalpetta (kerala) to Kozhikode,https://www.redbus.in/bus-tickets/kalpetta-to-...,A1 Travels,Bharat Benz A/C Sleeper (2+1),21:00,02h 30m,23:30,3.3,INR 500,30 Seats available
386,Kalpetta (kerala) to Kozhikode,https://www.redbus.in/bus-tickets/kalpetta-to-...,Cochin Express,A/C Seater Push Back (2+2),22:30,01h 25m,23:55,4.1,INR 700,41 Seats available
387,Kalpetta (kerala) to Kozhikode,https://www.redbus.in/bus-tickets/kalpetta-to-...,Cochin Express,A/C Seater Push Back (2+2),22:30,01h 25m,23:55,4.1,INR 700,41 Seats available
388,Kalpetta (kerala) to Kozhikode,https://www.redbus.in/bus-tickets/kalpetta-to-...,Sonia Travels,A/C Seater/Sleeper (2+1),21:30,01h 59m,23:29,3.8,INR 750,15 Seats available


In [48]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

# URL of the website
URL = "https://www.redbus.in/online-booking/ktcl/?utm_source=rtchometile"

def initialize_driver():
    driver = webdriver.Chrome()
    driver.maximize_window()
    return driver

def load_page(driver, url):
    driver.get(url)
    time.sleep(2)  # Wait for the page to load

# Function to scrape bus routes
def scrape_bus_routes(driver):
    route_elements = driver.find_elements(By.CLASS_NAME, 'route')
    bus_routes_link = [route.get_attribute('href') for route in route_elements]
    bus_routes_name = [route.text.strip() for route in route_elements]
    return bus_routes_link, bus_routes_name

# Function to scrape bus details
def scrape_bus_details(driver, url, route_name):
    try:
        driver.get(url)
        time.sleep(2)  # Allow the page to load
        
        # Click the "View Buses" button if it exists
        try:
            view_buses_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CLASS_NAME, "button"))
            )
            driver.execute_script("arguments[0].click();", view_buses_button)
            time.sleep(2)  # Wait for buses to load
            
            # Scroll down to load all bus items
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(2)  # Wait for the page to load more content

            # Find bus item details
            bus_name_elements = driver.find_elements(By.CLASS_NAME, "travels")
            bus_type_elements = driver.find_elements(By.CLASS_NAME, "bus-type")
            departing_time_elements = driver.find_elements(By.CLASS_NAME, "dp-time")
            duration_elements = driver.find_elements(By.CLASS_NAME, "dur")
            reaching_time_elements = driver.find_elements(By.CLASS_NAME, "bp-time")
            star_rating_elements = driver.find_elements(By.XPATH, "//div[@class='rating-sec lh-24']")
            price_elements = driver.find_elements(By.CLASS_NAME, "fare")

            # Use XPath to handle both seat availability classes
            seat_availability_elements = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left m-top-30') or contains(@class, 'seat-left m-top-16')]")

            bus_details = []
            for i in range(len(bus_name_elements)):
                bus_detail = {
                    "Route_Name": route_name,
                    "Route_Link": url,
                    "Bus_Name": bus_name_elements[i].text,
                    "Bus_Type": bus_type_elements[i].text,
                    "Departing_Time": departing_time_elements[i].text,
                    "Duration": duration_elements[i].text,
                    "Reaching_Time": reaching_time_elements[i].text,
                    "Star_Rating": star_rating_elements[i].text if i < len(star_rating_elements) else '0',
                     "Price": price_elements[i].text,
                    "Seat_Availability": seat_availability_elements[i].text if i < len(seat_availability_elements) else '0'
                }
                bus_details.append(bus_detail)
            return bus_details
        
        except Exception as e:
            print(f"Error occurred while scraping bus details for {url}: {str(e)}")
            return []

    except Exception as e:
        print(f"Error occurred while accessing {url}: {str(e)}")
        return []

# List to hold all bus details
all_bus_details = []

# Function to scrape all pages
def scrape_all_pages():
    for page in range(1, 5):  # There are 4 pages
        try:
            driver = initialize_driver()
            load_page(driver, URL)
            
            if page > 1:
                pagination_tab = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs')][text()='{page}']"))
                )
                driver.execute_script("arguments[0].scrollIntoView();", pagination_tab)
                driver.execute_script("arguments[0].click();", pagination_tab)
                time.sleep(5)  # Wait for the page to load
            
            all_bus_routes_link, all_bus_routes_name = scrape_bus_routes(driver)
            # Iterate over each bus route link and scrape the details
            for link, name in zip(all_bus_routes_link, all_bus_routes_name):
                bus_details = scrape_bus_details(driver, link, name)
                if bus_details:
                    all_bus_details.extend(bus_details)
            

        except Exception as e:
            print(f"Error occurred while accessing page {page}: {str(e)}")

# Scrape routes and details from all pages
scrape_all_pages()


# Convert the list of dictionaries to a DataFrame
df = pd.DataFrame(all_bus_details)

# Save the DataFrame to a CSV file
df.to_csv('kadamba_bus_details.csv', index=False)

# Close the driver
driver.quit()


Error occurred while scraping bus details for https://www.redbus.in/bus-tickets/calangute-goa-to-mopa-airport: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF6CB839642+30946]
	(No symbol) [0x00007FF6CB7EE3D9]
	(No symbol) [0x00007FF6CB6E6FDA]
	(No symbol) [0x00007FF6CB73822C]
	(No symbol) [0x00007FF6CB73850C]
	(No symbol) [0x00007FF6CB77DCB7]
	(No symbol) [0x00007FF6CB75CAAF]
	(No symbol) [0x00007FF6CB77B041]
	(No symbol) [0x00007FF6CB75C813]
	(No symbol) [0x00007FF6CB72A6E5]
	(No symbol) [0x00007FF6CB72B021]
	GetHandleVerifier [0x00007FF6CB96F84D+1301229]
	GetHandleVerifier [0x00007FF6CB97BDC7+1351783]
	GetHandleVerifier [0x00007FF6CB972A13+1313971]
	GetHandleVerifier [0x00007FF6CB86DD16+245686]
	(No symbol) [0x00007FF6CB7F759F]
	(No symbol) [0x00007FF6CB7F3814]
	(No symbol) [0x00007FF6CB7F39A2]
	(No symbol) [0x00007FF6CB7EA3FF]
	BaseThreadInitThunk [0x00007FFBF4AE257D+29]
	RtlUserThreadStart [0x00007FFBF642AF28+40]

Error occurred while scraping bus details for https://www.redbus

In [49]:
df

,Route_Name,Route_Link,Bus_Name,Bus_Type,Departing_Time,Duration,Reaching_Time,Star_Rating,Price,Seat_Availability
0,Pune to Goa,https://www.redbus.in/bus-tickets/pune-to-goa,Kadamba Transport Corporation Limited (KTCL) -...,A/C Sleeper (2+1),19:00,12h 10m,07:10,3.8,INR 1000,3 Seats available
1,Pune to Goa,https://www.redbus.in/bus-tickets/pune-to-goa,Ashray Travels,Bharat Benz A/C Sleeper (2+1),18:30,14h 00m,08:30,4.5,INR 750,15 Seats available
2,Pune to Goa,https://www.redbus.in/bus-tickets/pune-to-goa,Ashray Amrutyog,A/C Sleeper (2+1),20:15,13h 15m,09:30,4.4,INR 660,19 Seats available
3,Pune to Goa,https://www.redbus.in/bus-tickets/pune-to-goa,AdIntrCity SmartBus,Bharat Benz A/C Sleeper (2+1),21:45,11h 25m,09:10,3.9,INR 882,6 Seats available
4,Pune to Goa,https://www.redbus.in/bus-tickets/pune-to-goa,Ashray Vip Yellows,A/C Sleeper (2+1),18:25,13h 35m,08:00,4.2,INR 800,13 Seats available
...,...,...,...,...,...,...,...,...,...,...
235,Goa to Satara,https://www.redbus.in/bus-tickets/goa-to-satara,Giroba Travels (Kelkar),Non A/C Seater / Sleeper (2+1),18:45,09h 45m,04:30,3.3,INR 399,24 Seats available
236,Goa to Satara,https://www.redbus.in/bus-tickets/goa-to-satara,VRL Travels,A/C Sleeper (2+1),19:30,08h 30m,04:00,3.4,INR 1000,16 Seats available
237,Goa to Shivamogga,https://www.redbus.in/bus-tickets/goa-to-shimoga,Sea Bird Tourist,NON A/C Sleeper (2+1),18:30,08h 00m,02:30,3.1,INR 700,27 Seats available
238,Goa to Shirdi,https://www.redbus.in/bus-tickets/goa-to-shirdi,PSR Travels Goa,A/C Sleeper (2+1),19:15,14h 45m,10:00,3.6,INR 1199,11 Seats available


In [50]:
#Uttrapradesh
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

# URL of the website
URL = "https://www.redbus.in/online-booking/uttar-pradesh-state-road-transport-corporation-upsrtc/?utm_source=rtchometile"

def initialize_driver():
    driver = webdriver.Chrome()
    driver.maximize_window()
    return driver

def load_page(driver, url):
    driver.get(url)
    time.sleep(2)  # Wait for the page to load

# Function to scrape bus routes
def scrape_bus_routes(driver):
    route_elements = driver.find_elements(By.CLASS_NAME, 'route')
    bus_routes_link = [route.get_attribute('href') for route in route_elements]
    bus_routes_name = [route.text.strip() for route in route_elements]
    return bus_routes_link, bus_routes_name

# Function to scrape bus details
def scrape_bus_details(driver, url, route_name):
    try:
        driver.get(url)
        time.sleep(2)  # Allow the page to load
        
        # Click the "View Buses" button if it exists
        try:
            view_buses_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CLASS_NAME, "button"))
            )
            driver.execute_script("arguments[0].click();", view_buses_button)
            time.sleep(2)  # Wait for buses to load
            
            # Scroll down to load all bus items
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(2)  # Wait for the page to load more content

            # Find bus item details
            bus_name_elements = driver.find_elements(By.CLASS_NAME, "travels")
            bus_type_elements = driver.find_elements(By.CLASS_NAME, "bus-type")
            departing_time_elements = driver.find_elements(By.CLASS_NAME, "dp-time")
            duration_elements = driver.find_elements(By.CLASS_NAME, "dur")
            reaching_time_elements = driver.find_elements(By.CLASS_NAME, "bp-time")
            star_rating_elements = driver.find_elements(By.XPATH, "//div[@class='rating-sec lh-24']")
            price_elements = driver.find_elements(By.CLASS_NAME, "fare")

            # Use XPath to handle both seat availability classes
            seat_availability_elements = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left m-top-30') or contains(@class, 'seat-left m-top-16')]")

            bus_details = []
            for i in range(len(bus_name_elements)):
                bus_detail = {
                    "Route_Name": route_name,
                    "Route_Link": url,
                    "Bus_Name": bus_name_elements[i].text,
                    "Bus_Type": bus_type_elements[i].text,
                    "Departing_Time": departing_time_elements[i].text,
                    "Duration": duration_elements[i].text,
                    "Reaching_Time": reaching_time_elements[i].text,
                    "Star_Rating": star_rating_elements[i].text if i < len(star_rating_elements) else '0',
                    "Price": price_elements[i].text,
                    "Seat_Availability": seat_availability_elements[i].text if i < len(seat_availability_elements) else '0'
                }
                bus_details.append(bus_detail)
            return bus_details
        
        except Exception as e:
            print(f"Error occurred while scraping bus details for {url}: {str(e)}")
            return []

    except Exception as e:
        print(f"Error occurred while accessing {url}: {str(e)}")
        return []

# List to hold all bus details
all_bus_details = []

# Function to scrape all pages
def scrape_all_pages():
    for page in range(1, 6):  # There are 5 pages
        try:
            driver = initialize_driver()
            load_page(driver, URL)
            
            if page > 1:
                pagination_tab = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs')][text()='{page}']"))
                )
                driver.execute_script("arguments[0].scrollIntoView();", pagination_tab)
                driver.execute_script("arguments[0].click();", pagination_tab)
                time.sleep(5)  # Wait for the page to load
            
            all_bus_routes_link, all_bus_routes_name = scrape_bus_routes(driver)
            # Iterate over each bus route link and scrape the details
            for link, name in zip(all_bus_routes_link, all_bus_routes_name):
                bus_details = scrape_bus_details(driver, link, name)
                if bus_details:
                    all_bus_details.extend(bus_details)


        except Exception as e:
            print(f"Error occurred while accessing page {page}: {str(e)}")

# Scrape routes and details from all pages
scrape_all_pages()

# Convert the list of dictionaries to a DataFrame
df = pd.DataFrame(all_bus_details)

# Save the DataFrame to a CSV file
df.to_csv('up_bus_details.csv', index=False)

# Close the driver
driver.quit()

Error occurred while scraping bus details for https://www.redbus.in/bus-tickets/bareilly-to-agra: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF6CB839642+30946]
	(No symbol) [0x00007FF6CB7EE3D9]
	(No symbol) [0x00007FF6CB6E6FDA]
	(No symbol) [0x00007FF6CB73822C]
	(No symbol) [0x00007FF6CB73850C]
	(No symbol) [0x00007FF6CB77DCB7]
	(No symbol) [0x00007FF6CB75CAAF]
	(No symbol) [0x00007FF6CB77B041]
	(No symbol) [0x00007FF6CB75C813]
	(No symbol) [0x00007FF6CB72A6E5]
	(No symbol) [0x00007FF6CB72B021]
	GetHandleVerifier [0x00007FF6CB96F84D+1301229]
	GetHandleVerifier [0x00007FF6CB97BDC7+1351783]
	GetHandleVerifier [0x00007FF6CB972A13+1313971]
	GetHandleVerifier [0x00007FF6CB86DD16+245686]
	(No symbol) [0x00007FF6CB7F759F]
	(No symbol) [0x00007FF6CB7F3814]
	(No symbol) [0x00007FF6CB7F39A2]
	(No symbol) [0x00007FF6CB7EA3FF]
	BaseThreadInitThunk [0x00007FFBF4AE257D+29]
	RtlUserThreadStart [0x00007FFBF642AF28+40]

Error occurred while scraping bus details for https://www.redbus.in/bus-ticke

In [51]:
df

,Route_Name,Route_Link,Bus_Name,Bus_Type,Departing_Time,Duration,Reaching_Time,Star_Rating,Price,Seat_Availability
0,Bareilly to Delhi,https://www.redbus.in/bus-tickets/bareilly-to-...,UPSRTC - HDD0063,Ordinary Non AC Seater 2+3,19:39,05h 24m,01:03,3.3,INR 415,52 Seats available
1,Bareilly to Delhi,https://www.redbus.in/bus-tickets/bareilly-to-...,UPSRTC - RKD0128,Janrath AC Seater 2+2,20:01,05h 00m,01:01,4.1,INR 598,36 Seats available
2,Bareilly to Delhi,https://www.redbus.in/bus-tickets/bareilly-to-...,UPSRTC - LNI0017,Rajdhani Non AC Seater 2+3,21:31,05h 29m,03:00,3.3,INR 418,51 Seats available
3,Bareilly to Delhi,https://www.redbus.in/bus-tickets/bareilly-to-...,UPSRTC - BLY0139,Janrath AC Seater 2+2,21:31,05h 59m,03:30,3.1,INR 598,36 Seats available
4,Bareilly to Delhi,https://www.redbus.in/bus-tickets/bareilly-to-...,UPSRTC - RKD0121,Shatabdi AC Seater 2+2,22:01,05h 00m,03:01,3.2,INR 598,37 Seats available
...,...,...,...,...,...,...,...,...,...,...
481,Delhi to Haldwani,https://www.redbus.in/bus-tickets/delhi-to-hal...,ROYAL SAFARI INDIA TRAVELS,Bharat Benz A/C Semi Sleeper (2+2),22:00,07h 30m,05:30,2.4,899,30 Seats available
482,Delhi to Haldwani,https://www.redbus.in/bus-tickets/delhi-to-hal...,ROYAL SAFARI INDIA TRAVELS,Bharat Benz A/C Seater /Sleeper (2+1),22:30,07h 30m,06:00,1.7,899,21 Seats available
483,Delhi to Haldwani,https://www.redbus.in/bus-tickets/delhi-to-hal...,Travel Express,AC Seater (2+2),23:30,04h 30m,04:00,0,INR 951.43,45 Seats available
484,Delhi to Haldwani,https://www.redbus.in/bus-tickets/delhi-to-hal...,Aaikaa Travels Pvt Ltd,A/C Seater (2+2),22:15,07h 45m,06:00,0,INR 1300,18 Seats available
